<a href="https://colab.research.google.com/github/sihan827/2021_Winter_PE/blob/main/colab_model/cifar10_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torchvision import utils, datasets, transforms
from torch.utils import data

In [16]:
batch_size = 10

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_set = datasets.CIFAR10(
    '/content/drive/My Drive/.data',
    train=True,
    download=True,
    transform=preprocess
)

test_set = datasets.CIFAR10(
    '/content/drive/My Drive/.data',
    train=False,
    download=True,
    transform=preprocess
)

train_loader = data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = data.DataLoader(
    dataset=test_set,
    batch_size=batch_size,
    num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
def train(model):
  model.train()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  if torch.cuda.is_available():
    model.to('cuda')
  for epoch in range(8):
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader, 0):
      if torch.cuda.is_available():
        data, target = data.to('cuda'), target.to('cuda')
        # print(data.shape)
        optimizer.zero_grad()

        output = model(data)
        loss = torch.nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        # print(output)
        # print(loss)
        
        # break

        running_loss += loss.item()
        if batch_idx % 1000 == 999:
          print('epoch {}, batch {}, loss: {}'.format(epoch, batch_idx, running_loss/1000))
          running_loss = 0
  print('done!')

In [19]:
def test(model):
  model.eval()
  if torch.cuda.is_available():
    model.to('cuda')
  total = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      if torch.cuda.is_available():
        data, target = data.to('cuda'), target.to('cuda')
        output = model(data)

        pred = output.max(1, keepdim=True)[1]
        pred = torch.squeeze(pred)
        total += target.size(0)
        correct += (pred == target).sum().item()

  print('Accuracy in test data(10000):', 100 * correct / total, '%')



In [20]:
alexnet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
alexnet_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
alexnet_model.classifier[4] = torch.nn.Linear(4096, 1024)
alexnet_model.classifier[6] = torch.nn.Linear(1024, 10)
alexnet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [40]:
train(alexnet_model)

epoch 0, batch 999, loss: 0.1205517328629794
epoch 0, batch 1999, loss: 0.13354026320978482
epoch 0, batch 2999, loss: 0.14324792468479972
epoch 0, batch 3999, loss: 0.13889610375252232
epoch 0, batch 4999, loss: 0.14722705642829534
epoch 1, batch 999, loss: 0.09455011963797916
epoch 1, batch 1999, loss: 0.11699619505125884
epoch 1, batch 2999, loss: 0.12108273787769212
epoch 1, batch 3999, loss: 0.12494278746318013
epoch 1, batch 4999, loss: 0.1419955288547394
epoch 2, batch 999, loss: 0.102316243734017
epoch 2, batch 1999, loss: 0.10050106644192557
epoch 2, batch 2999, loss: 0.09130935081523239
epoch 2, batch 3999, loss: 0.10756797730477047
epoch 2, batch 4999, loss: 0.11010165659112398
epoch 3, batch 999, loss: 0.0876033715325907
epoch 3, batch 1999, loss: 0.09132270540201216
epoch 3, batch 2999, loss: 0.09744842938410966
epoch 3, batch 3999, loss: 0.10874556620729073
epoch 3, batch 4999, loss: 0.10009178847103621
epoch 4, batch 999, loss: 0.07334535301766118
epoch 4, batch 1999, lo

In [41]:
test(alexnet_model)

Accuracy in test data(10000): 87.9 %


In [26]:
vgg11_model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)
vgg11_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [27]:
vgg11_model.classifier[3] = torch.nn.Linear(4096, 1024)
vgg11_model.classifier[6] = torch.nn.Linear(1024, 10)
vgg11_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [28]:
train(vgg11_model)

epoch 0, batch 999, loss: 0.9537251698002219
epoch 0, batch 1999, loss: 0.5952465439103544
epoch 0, batch 2999, loss: 0.5328426087386906
epoch 0, batch 3999, loss: 0.4910727524673566
epoch 0, batch 4999, loss: 0.4485982703557238
epoch 1, batch 999, loss: 0.3171060707292054
epoch 1, batch 1999, loss: 0.3150876182403299
epoch 1, batch 2999, loss: 0.32116765585169194
epoch 1, batch 3999, loss: 0.3082453236381989
epoch 1, batch 4999, loss: 0.30836104781972246
epoch 2, batch 999, loss: 0.18620348133627704
epoch 2, batch 1999, loss: 0.20219622290341066
epoch 2, batch 2999, loss: 0.1953517668845743
epoch 2, batch 3999, loss: 0.2070796354438935
epoch 2, batch 4999, loss: 0.21788699717458804
epoch 3, batch 999, loss: 0.12127013285095745
epoch 3, batch 1999, loss: 0.12736824477365008
epoch 3, batch 2999, loss: 0.14314014584887264
epoch 3, batch 3999, loss: 0.13852102207766073
epoch 3, batch 4999, loss: 0.1538130817049823
epoch 4, batch 999, loss: 0.07325584194664861
epoch 4, batch 1999, loss: 0.

In [29]:
test(vgg11_model)

Accuracy in test data(10000): 91.11 %


In [31]:
resnet_model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
resnet_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
resnet_model.fc = torch.nn.Linear(512, 10)
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
train(resnet_model)

epoch 0, batch 999, loss: 0.7929994599148631
epoch 0, batch 1999, loss: 0.502712885858491
epoch 0, batch 2999, loss: 0.4306517798602581
epoch 0, batch 3999, loss: 0.39222407049080354
epoch 0, batch 4999, loss: 0.3560291937626898
epoch 1, batch 999, loss: 0.24043052234314383
epoch 1, batch 1999, loss: 0.25181208456959575
epoch 1, batch 2999, loss: 0.23920279976772144
epoch 1, batch 3999, loss: 0.2508164566467749
epoch 1, batch 4999, loss: 0.2417995095390361
epoch 2, batch 999, loss: 0.14401165513077285
epoch 2, batch 1999, loss: 0.1436221687712241
epoch 2, batch 2999, loss: 0.15933692928391974
epoch 2, batch 3999, loss: 0.16020406106345764
epoch 2, batch 4999, loss: 0.15050782316509867
epoch 3, batch 999, loss: 0.0992554651676328
epoch 3, batch 1999, loss: 0.10328785258141579
epoch 3, batch 2999, loss: 0.1056096562181774
epoch 3, batch 3999, loss: 0.09692003037751419
epoch 3, batch 4999, loss: 0.12360551182588096
epoch 4, batch 999, loss: 0.06565931338298833
epoch 4, batch 1999, loss: 0

In [34]:
test(resnet_model)

Accuracy in test data(10000): 93.84 %


In [36]:
googlenet_model = torch.hub.load('pytorch/vision:v0.6.0', 'googlenet', pretrained=True)
googlenet_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [37]:
googlenet_model.fc = torch.nn.Linear(1024, 10)
googlenet_model.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [38]:
train(googlenet_model)

epoch 0, batch 999, loss: 0.9799028879925609
epoch 0, batch 1999, loss: 0.5242626264300198
epoch 0, batch 2999, loss: 0.4537823651023209
epoch 0, batch 3999, loss: 0.4058942521251738
epoch 0, batch 4999, loss: 0.36816893765330316
epoch 1, batch 999, loss: 0.2858906963663176
epoch 1, batch 1999, loss: 0.2782049632556736
epoch 1, batch 2999, loss: 0.26361650348920374
epoch 1, batch 3999, loss: 0.2635554883959703
epoch 1, batch 4999, loss: 0.2502821253789589
epoch 2, batch 999, loss: 0.18011765402765012
epoch 2, batch 1999, loss: 0.1842700562274549
epoch 2, batch 2999, loss: 0.19825646942318417
epoch 2, batch 3999, loss: 0.17575354475015775
epoch 2, batch 4999, loss: 0.18202206104970536
epoch 3, batch 999, loss: 0.1302790405472042
epoch 3, batch 1999, loss: 0.13665165859140688
epoch 3, batch 2999, loss: 0.13633199337520638
epoch 3, batch 3999, loss: 0.12947894353128503
epoch 3, batch 4999, loss: 0.1397654125854315
epoch 4, batch 999, loss: 0.09239062660877244
epoch 4, batch 1999, loss: 0.

In [39]:
test(googlenet_model)

Accuracy in test data(10000): 94.04 %
